In [1]:
%%bash
pwd
pip install yfinance
pip install ta

/content
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29103 sha256=3c287befc306af8ee84c2e75bf8d640629ca829636c97d20bf590dcecd880a78
  Stored in directory: /root/.cache/pip/wheels/6a/76/03/7f785aaa50b9c6ec7e3fd105a62c1b2c45a034512d51e024a9
Successfully built ta


In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
ticker_data = yf.download(tickers = "MRK",  # list of tickers
            start="2009-01-01", end="2020-01-01",       # time period
            interval = "1d",       # trading interval
            ignore_tz = True,      # ignore timezone when aligning data from different exchanges?
            prepost = False) 
ticker_data.index = pd.DatetimeIndex(ticker_data.index).to_period('D')
print(ticker_data.shape)
ticker_data.head()

[*********************100%***********************]  1 of 1 completed
(2768, 6)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-01-02,29.064884,29.694656,28.921757,29.580153,17.823135,12793355
2009-01-05,30.009542,30.209925,28.902672,29.131680,17.552916,15686988
2009-01-06,29.656488,29.828243,28.196566,28.597328,17.230949,15752698
2009-01-07,28.253817,28.511450,27.824427,28.139313,16.954981,13325530
2009-01-08,28.320610,28.330153,27.709925,28.015266,16.880238,11092137


In [3]:
import numpy as np
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from sklearn.metrics import mean_squared_error

def time_series_cross_validation(data, alpha_values):
    train_size = int(len(data) * 0.8)
    train, validation = data[:train_size], data[train_size:]
    
    best_alpha = None
    best_mse = np.inf
    
    for alpha in alpha_values:
        smoothing_model = SimpleExpSmoothing(train, initialization_method="heuristic").fit(smoothing_level=alpha, optimized=False)
        predictions = smoothing_model.forecast(len(validation))
        mse = mean_squared_error(validation, predictions)
        
        if mse < best_mse:
            best_mse = mse
            best_alpha = alpha
            
    return best_alpha, best_mse

# Test different alpha values
alpha_values = np.linspace(0.0001, 1, 1000)

# Assume 'Close' column contains the closing prices of the stock
best_alpha, best_mse = time_series_cross_validation(ticker_data['Close'], alpha_values)

print(f"Best alpha value: {best_alpha}, with MSE: {best_mse}")

Best alpha value: 0.07016306306306307, with MSE: 169.1034863488541


In [4]:
# Choose the smoothing factor (alpha) based on the cross-validation results
alpha = best_alpha

def apply_smoothing(column, alpha):
    smoothing_model = SimpleExpSmoothing(column, initialization_method="heuristic")
    smoothing_model = smoothing_model.fit(optimized=True)
    return smoothing_model.fittedvalues

# Apply exponential smoothing to all numeric columns
smoothed_data = ticker_data.select_dtypes(include=[np.number]).apply(apply_smoothing, alpha=alpha)

# Add the smoothed values to the DataFrame as new columns
for col in smoothed_data.columns:
    ticker_data[col] = smoothed_data[col]

ticker_data.sample(20)

/usr/local/lib/python3.9/dist-packages/statsmodels/tsa/holtwinters/model.py:915: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2016-10-24,58.739662,59.064884,58.234734,58.396946,47.792171,8.433302e+06
2016-07-15,57.125665,57.185116,56.755726,56.917938,46.242657,8.189638e+06
2009-08-11,28.890101,29.580153,28.730915,29.198473,18.137877,1.593454e+07
2018-11-14,71.773210,71.822517,70.620232,71.192749,61.931698,1.550778e+07
2012-02-01,37.172515,37.204197,36.440842,36.517178,25.346031,1.487674e+07
2016-09-22,59.400916,59.942749,59.026718,59.828243,48.963543,9.640370e+06
2016-11-14,61.154667,61.555344,60.667938,61.020992,49.939690,1.672670e+07
2009-10-22,32.207270,32.251907,31.116411,31.183207,19.605124,2.508599e+07
2019-09-04,82.309086,83.024811,82.099236,82.681297,73.408485,6.690904e+06


In [5]:
# Generate technical indicators using the TA-Lib library
from ta.utils import dropna
import ta
import pandas as pd

ticker_data = dropna(ticker_data)

# List of technical indicators to calculate
indicators = ['SMA', 'EMA', 'MACD', 'RSI', 'ADX', 'BollingerBands', 'Stochastic_Oscillator', 'Williams_R', 'Price_ROC', 'OBV']

# Create a new DataFrame to store the technical indicators
technical_indicators = pd.DataFrame(index=ticker_data.index)

# Calculate technical indicators
technical_indicators['SMA7'] = ta.trend.SMAIndicator(close=ticker_data['Close'], window=7).sma_indicator()
technical_indicators['SMA21'] = ta.trend.SMAIndicator(close=ticker_data['Close'], window=21).sma_indicator()
technical_indicators['EMA7'] = ta.trend.EMAIndicator(close=ticker_data['Close'], window=7).ema_indicator()
technical_indicators['EMA21'] = ta.trend.EMAIndicator(close=ticker_data['Close'], window=21).ema_indicator()

macd = ta.trend.MACD(close=ticker_data['Close']).macd()
macdsignal = ta.trend.MACD(close=ticker_data['Close']).macd_signal()
technical_indicators['MACD'] = macd
technical_indicators['MACD_signal'] = macdsignal

technical_indicators['RSI'] = ta.momentum.RSIIndicator(close=ticker_data['Close'], window=14).rsi()
technical_indicators['ADX'] = ta.trend.ADXIndicator(high=ticker_data['High'], low=ticker_data['Low'], close=ticker_data['Close'], window=14).adx()

technical_indicators['BB_upper'], technical_indicators['BB_middle'], technical_indicators['BB_lower'] = ta.volatility.BollingerBands(close=ticker_data['Close'], window=20).bollinger_mavg(), ta.volatility.BollingerBands(close=ticker_data['Close'], window=20).bollinger_hband(), ta.volatility.BollingerBands(close=ticker_data['Close'], window=20).bollinger_lband()

# Adding Stochastic Oscillator, Williams %R, Price Rate of Change, and On Balance Volume
# Stochastic Oscillator has a window of 14 days
technical_indicators['Stochastic_Oscillator'] = ta.momentum.StochasticOscillator(high=ticker_data['High'], low=ticker_data['Low'], close=ticker_data['Close'], window=14).stoch()
# Williams %R has a window of 14 days
technical_indicators['Williams_R'] = ta.momentum.WilliamsRIndicator(high=ticker_data['High'], low=ticker_data['Low'], close=ticker_data['Close'], lbp=14).williams_r()
technical_indicators['Price_ROC'] = ta.momentum.ROCIndicator(close=ticker_data['Close'], window=12).roc()
technical_indicators['OBV'] = ta.volume.OnBalanceVolumeIndicator(close=ticker_data['Close'], volume=ticker_data['Volume']).on_balance_volume()


/usr/local/lib/python3.9/dist-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/usr/local/lib/python3.9/dist-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


In [6]:
technical_indicators = technical_indicators.dropna()
technical_indicators.reset_index(inplace=True)
technical_indicators.rename(columns={'index': 'Datetime'}, inplace=True)
technical_indicators.sample(10)

,Date,SMA7,SMA21,EMA7,EMA21,MACD,MACD_signal,RSI,ADX,BB_upper,BB_middle,BB_lower,Stochastic_Oscillator,Williams_R,Price_ROC,OBV
2345,2018-06-14,59.330698,57.572246,59.216002,57.915029,1.082006,0.889230,67.979217,21.893125,57.626431,60.549600,54.703262,92.494920,-7.505080,5.923169,4.822064e+08
1343,2014-06-23,55.657034,55.216739,55.731165,55.262691,0.429870,0.400125,63.516870,24.594447,55.282443,56.484179,54.080707,82.353022,-17.646978,1.708959,2.317549e+08
2269,2018-02-26,52.637676,54.394310,52.566197,53.769064,-0.983050,-0.796668,38.749156,32.511286,54.189408,58.902193,49.476622,27.012489,-72.987511,-1.063829,4.091225e+08
383,2010-08-27,33.117502,33.359233,33.093498,33.334487,-0.159546,-0.093566,44.087653,8.822104,33.354485,34.054439,32.654530,23.350400,-76.649600,-3.606371,-1.268876e+08
1268,2014-03-06,54.006270,53.010269,54.004246,52.936250,1.087211,1.108570,66.247921,43.466115,53.107824,54.935189,51.280459,80.983696,-19.016304,2.597404,2.216779e+08
1089,2013-06-19,45.620229,45.441657,45.651711,45.408269,0.337298,0.417162,52.639210,23.662258,45.556775,46.920688,44.192861,34.332421,-65.667579,2.248392,2.171153e+08
2493,2019-01-16,71.439477,71.269084,71.219703,71.589151,-0.222547,-0.117826,47.868247,18.035277,71.183683,73.559913,68.807453,61.605569,-38.394431,-1.167424,5.930860e+08
2223,2017-12-18,53.559161,52.830335,53.575303,53.419415,-0.239040,-0.580445,49.994190,24.957531,52.839695,54.531050,51.148340,64.465444,-35.534556,1.589596,4.704819e+08
598,2011-07-06,33.461015,33.740458,33.670427,33.804764,-0.170644,-0.206656,50.109217,13.791652,33.730916,34.471685,32.990148,70.000175,-29.999825,0.881187,-8.908844e+07
1002,2013-02-13,39.310250,40.463014,39.588201,40.152458,-0.427225,-0.299247,42.125576,23.436489,40.418416,42.205682,38.631150,21.088408,-78.911592,-4.624950,2.945653e+08


In [7]:
# Perform normalization
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
technical_indicators_scaled = pd.DataFrame(scaler.fit_transform(technical_indicators.drop(columns=['Date'])), columns=technical_indicators.drop(columns=['Date']).columns)

technical_indicators_scaled

,SMA7,SMA21,EMA7,EMA21,MACD,MACD_signal,RSI,ADX,BB_upper,BB_middle,BB_lower,Stochastic_Oscillator,Williams_R,Price_ROC,OBV
0,0.096126,0.069330,0.092274,0.068477,0.556402,0.579424,0.469450,0.282728,0.071184,0.070880,0.111116,0.213115,0.213115,0.466202,0.124502
1,0.094239,0.069718,0.089231,0.067013,0.539614,0.568838,0.423723,0.301514,0.071683,0.069930,0.113019,0.098726,0.098726,0.336883,0.112373
2,0.090425,0.070032,0.086144,0.065374,0.522640,0.556568,0.401396,0.321970,0.071437,0.070265,0.112213,0.078550,0.078550,0.342469,0.099225
3,0.088746,0.070274,0.086206,0.064795,0.523111,0.546857,0.497992,0.334492,0.072004,0.069619,0.113951,0.292035,0.292035,0.369276,0.112954
4,0.087004,0.070362,0.083985,0.063400,0.511849,0.536566,0.432332,0.346120,0.071652,0.070049,0.112843,0.109145,0.109145,0.298270,0.097626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2730,0.983022,0.985941,0.990862,0.987253,0.950099,0.936667,0.929221,0.354714,0.986605,0.989079,0.984737,0.835165,0.835165,0.518728,1.000000
2731,0.987686,0.990325,0.994767,0.991085,0.958328,0.948408,0.883508,0.376886,0.991003,0.991294,0.991100,0.743040,0.743040,0.510825,0.993864
2732,0.991936,0.994452,0.997331,0.994428,0.958582,0.957858,0.868047,0.397473,0.994069,0.995142,0.993263,0.707206,0.707206,0.511488,0.989068
2733,0.997077,0.997497,0.999838,0.997692,0.957785,0.965239,0.877808,0.416591,0.997050,0.998943,0.995309,0.743242,0.743242,0.506970,0.993208


In [8]:
from google.colab import drive
drive.mount('/content/drive')

# Save the data to CSV files
technical_indicators_scaled.to_csv('/content/drive/My Drive/technical_indicators_scaled_day.csv', index=False)
ticker_data.to_csv('/content/drive/My Drive/ticker_data_day.csv', index=False)

Mounted at /content/drive
